In [17]:
import sys
from glob import glob

import time
import h5py
import zarr
import pygrib
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du

from datetime import datetime, timedelta

import dask.array as da

In [18]:
from scipy.spatial import cKDTree

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
# deep learning tools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras import Model

2023-03-06 09:26:57.109640: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [49]:
class LayerScale(layers.Layer):
    """Layer scale module.
    References:
      - https://arxiv.org/abs/2103.17239
    Args:
      init_values (float): Initial value for layer scale. Should be within
        [0, 1].
      projection_dim (int): Projection dimensionality.
    Returns:
      Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, input_shape):
        self.gamma = tf.Variable(
            self.init_values * tf.ones((self.projection_dim,))
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config
    
def create_model(input_shape=(64, 64, 15)):

    depths=[3, 3, 27, 3]
    projection_dims=[32, 64, 96, 128]
    drop_path_rate=0.0
    layer_scale_init_value=1e-6


    model_name='Branch64X'
    IN64 = layers.Input(shape=input_shape)
    X = IN64
    # ----- convnext block 0 ----- #

    X = layers.Conv2D(projection_dims[0], kernel_size=4, strides=4, name="{}_down0".format(model_name))(X)
    X = layers.LayerNormalization(epsilon=1e-6, name="{}_down0_norm".format(model_name))(X)

    for j in range(depths[0]):

        X_convnext = X
        X_convnext = layers.Conv2D(filters=projection_dims[0], kernel_size=7, padding="same",
                                   groups=projection_dims[0], name="{}_down0_dconv{}".format(model_name, j))(X_convnext)
        X_convnext = layers.LayerNormalization(epsilon=1e-6, name="{}_down0_dconv{}_norm".format(model_name, j))(X_convnext)
        X_convnext = layers.Dense(4 * projection_dims[0], name="{}_down0_dense{}_p1".format(model_name, j))(X_convnext)
        X_convnext = layers.Activation("gelu", name="{}_down0_gelu{}".format(model_name, j))(X_convnext)
        X_convnext = layers.Dense(projection_dims[0], name="{}_down0_dense{}_p2".format(model_name, j))(X_convnext)

        X_convnext = LayerScale(layer_scale_init_value, projection_dims[0], name="{}_down0_layerscale{}".format(model_name, j))(X_convnext)

        X = X + X_convnext


    # ----- convnext block 1 ----- #

    X = layers.LayerNormalization(epsilon=1e-6, name="{}_down1_norm".format(model_name))(X)
    X = layers.Conv2D(projection_dims[1], kernel_size=2, strides=2, name="{}_down1".format(model_name))(X)

    for j in range(depths[1]):

        X_convnext = X
        X_convnext = layers.Conv2D(filters=projection_dims[1], kernel_size=7, padding="same",
                                   groups=projection_dims[1], name="{}_down1_dconv{}".format(model_name, j))(X_convnext)
        X_convnext = layers.LayerNormalization(epsilon=1e-6, name="{}_down1_dconv{}_norm".format(model_name, j))(X_convnext)
        X_convnext = layers.Dense(4 * projection_dims[1], name="{}_down1_dense{}_p1".format(model_name, j))(X_convnext)
        X_convnext = layers.Activation("gelu", name="{}_down1_gelu{}".format(model_name, j))(X_convnext)
        X_convnext = layers.Dense(projection_dims[1], name="{}_down1_dense{}_p2".format(model_name, j))(X_convnext)

        X_convnext = LayerScale(layer_scale_init_value, projection_dims[1], name="{}_down1_layerscale{}".format(model_name, j))(X_convnext)

        X = X + X_convnext

    # ----- convnext block 2 ----- #

    X = layers.LayerNormalization(epsilon=1e-6, name="{}_down2_norm".format(model_name))(X)
    X = layers.Conv2D(projection_dims[2], kernel_size=2, strides=2, name="{}_down2".format(model_name))(X)

    for j in range(depths[2]):

        X_convnext = X
        X_convnext = layers.Conv2D(filters=projection_dims[2], kernel_size=5, padding="same",
                                   groups=projection_dims[2], name="{}_down2_dconv{}".format(model_name, j))(X_convnext)
        X_convnext = layers.LayerNormalization(epsilon=1e-6, name="{}_down2_dconv{}_norm".format(model_name, j))(X_convnext)
        X_convnext = layers.Dense(4 * projection_dims[2], name="{}_down2_dense{}_p1".format(model_name, j))(X_convnext)
        X_convnext = layers.Activation("gelu", name="{}_down2_gelu{}".format(model_name, j))(X_convnext)
        X_convnext = layers.Dense(projection_dims[2], name="{}_down2_dense{}_p2".format(model_name, j))(X_convnext)

        X_convnext = LayerScale(layer_scale_init_value, projection_dims[2], name="{}_down2_layerscale{}".format(model_name, j))(X_convnext)

        X = X + X_convnext

    # ----- convnext block 3 ----- #

    X = layers.LayerNormalization(epsilon=1e-6, name="{}_down3_norm".format(model_name))(X)
    X = layers.Conv2D(projection_dims[3], kernel_size=2, padding='same', name="{}_down3".format(model_name))(X)

    for j in range(depths[3]):

        X_convnext = X
        X_convnext = layers.Conv2D(filters=projection_dims[3], kernel_size=5, padding="same",
                                   groups=projection_dims[3], name="{}_down3_dconv{}".format(model_name, j))(X_convnext)
        X_convnext = layers.LayerNormalization(epsilon=1e-6, name="{}_down3_dconv{}_norm".format(model_name, j))(X_convnext)
        X_convnext = layers.Dense(4 * projection_dims[3], name="{}_down3_dense{}_p1".format(model_name, j))(X_convnext)
        X_convnext = layers.Activation("gelu", name="{}_down3_gelu{}".format(model_name, j))(X_convnext)
        X_convnext = layers.Dense(projection_dims[3], name="{}_down3_dense{}_p2".format(model_name, j))(X_convnext)

        X_convnext = LayerScale(layer_scale_init_value, projection_dims[3], name="{}_down3_layerscale{}".format(model_name, j))(X_convnext)

        X = X + X_convnext

    V1 = X

    OUT1 = layers.GlobalMaxPooling2D(name="{}_head_pool64".format(model_name))(V1)
    model = Model(inputs=IN64, outputs=OUT1, name=model_name)

    return model

In [ ]:
date_temp = datetime(2021, 1, 1, 0, 0)

In [20]:
with h5py.File(save_dir+'HRRRv4_STATS.hdf', 'r') as h5io:
    mean_stats = h5io['mean_stats'][...]
    std_stats = h5io['std_stats'][...]
    max_stats = h5io['max_stats'][...]

In [21]:
with h5py.File(save_dir+'HRRR_domain.hdf', 'r') as h5io:
    lon_3km = h5io['lon_3km'][...]
    lat_3km = h5io['lat_3km'][...]
    lon_80km = h5io['lon_80km'][...]
    lat_80km = h5io['lat_80km'][...]

In [22]:
LEADs = [[2, 3, 4], [3, 4, 5], [4, 5, 6], 
         [5, 6, 7], [6, 7, 8], [7, 8, 9], 
         [8, 9, 10], [9, 10, 11], [10, 11, 12], 
         [11, 12, 13], [12, 13, 14], [13, 14, 15], 
         [14, 15, 16], [15, 16, 17], [16, 17, 18], 
         [17, 18, 19], [18, 19, 20], [19, 20, 21], 
         [20, 21, 22], [21, 22, 23], [22, 23, 24]]

In [23]:
HRRR_dir = '/glade/campaign/cisl/aiml/ksha/HRRR/'

In [50]:
model_name = '/glade/work/ksha/NCAR/Keras_models/RE2_peak_base5/'

In [36]:

input_size = 64

log_norm = [True, False, True, True, True, False, False, True, True, True, True, False, False, False, False]
HRRRv4_inds = [1003, 1014, 1018, 1020, 1028, 1041, 1044, 1049, 1060, 1074, 1075, 1097, 1098, 1103, 1104]
var_names = ['1003:Maximum/Composite radar reflec',
             '1014:MSLP (MAPS System Reduction):P',
             '1018:199:199 (max):lambert:heightAb',
             '1020:199:199 (max):lambert:heightAb',
             '1028:74:74 (max):lambert:atmosphere',
             '1041:2 metre temperature:K (instant',
             '1044:2 metre dewpoint temperature:K',
             '1049:10 metre wind speed:m s**-1 (m',
             '1060:Total Precipitation:kg m**-2 (',
             '1074:Convective available potential',
             '1075:Convective inhibition:J kg**-1',
             '1097:Storm relative helicity:J kg**',
             '1098:Storm relative helicity:J kg**',
             '1103:Vertical u-component shear:s**',
             '1104:Vertical v-component shear:s**']

N_var = len(var_names)
half_margin = int(0.5*input_size)

In [ ]:
shape_80km = lon_80km.shape
shape_3km = lon_3km.shape

indx_array = np.empty(shape_80km)
indy_array = np.empty(shape_80km)

gridTree = cKDTree(list(zip(lon_3km.ravel(), lat_3km.ravel()))) #KDTree_wraper(xgrid, ygrid)

for xi in range(shape_80km[0]):
    for yi in range(shape_80km[1]):
        
        temp_lon = lon_80km[xi, yi]
        temp_lat = lat_80km[xi, yi]
        
        dist, indexes = gridTree.query(list(zip(np.array(temp_lon)[None], np.array(temp_lat)[None])))
        indx_3km, indy_3km = np.unravel_index(indexes, shape_3km)
        
        indx_array[xi, yi] = indx_3km[0]
        indy_array[xi, yi] = indy_3km[0]

In [ ]:
# Crerate model
model = create_model(input_shape=(64, 64, 15))

# get current weights
W_new = model.get_weights()

# get stored weights
W_old = k_utils.dummy_loader(model_name)

# update stored weights to new weights
for i in range(len(W_new)):
    if W_new[i].shape == W_old[i].shape:
        W_new[i] = W_old[i]

# dump new weights to the model
model.set_weights(W_new)

# compile just in case
model.compile(loss=keras.losses.mean_absolute_error, optimizer=keras.optimizers.SGD(lr=0))

In [94]:
VARs = np.empty((1059, 1799, 15))
VARs[...] = np.nan

FEATURE_VEC = np.empty((65, 93, 21, 128))
FEATURE_VEC[...] = np.nan

input_frame = np.empty((1, 64, 64, 15))
input_frame[...] = np.nan

for l in range(2):
    
    leads = LEADs[l]
    lead = leads[0]

    filename_grib = (datetime.strftime(date_temp, HRRR_dir+'fcst{:02d}hr/HRRR.%Y%m%d.natf{:02d}.grib2')).format(lead, lead)

    var_names_temp = []
    with pygrib.open(filename_grib) as grbio:
        for i, ind in enumerate(HRRRv4_inds):
            var_names_temp.append(str(grbio[ind])[:35])

    flag_qc = var_names == var_names_temp
    
    with pygrib.open(filename_grib) as grbio:
        for i, ind in enumerate(HRRRv4_inds):
            VARs[..., i] = grbio[ind].values
        
    for ix in range(shape_80km[0]):
        for iy in range(shape_80km[1]):

            indx = int(indx_array[ix, iy])
            indy = int(indy_array[ix, iy])

            x_edge_left = indx - half_margin
            x_edge_right = indx + half_margin

            y_edge_bottom = indy - half_margin
            y_edge_top = indy + half_margin

            if x_edge_left >= 0 and y_edge_bottom >= 0 and x_edge_right < shape_3km[0] and y_edge_top < shape_3km[1]:

                hrrr_temp = VARs[x_edge_left:x_edge_right, y_edge_bottom:y_edge_top, :]

                for n in range(N_var):

                    means = mean_stats[ix, iy, n, l]
                    stds = std_stats[ix, iy, n, l]
                    max_vals = max_stats[ix, iy, n, l]

                    temp = hrrr_temp[..., n]

                    # (n==0) Radar reflectivity, correct negative to 0
                    if n == 0:
                        temp[temp<0] = 0

                    # (n==10) CIN, preserve negative vals only, and convert them to positive 
                    if n == 10:
                        temp = -1*temp
                        temp[temp<0] = 0

                    # variables that will be normalizaed with log transformation
                    if log_norm[n]:
                        temp = np.log(np.abs(temp)+1)
                        # for CIN and SRH, x3 the value
                        if n < 9:
                            temp = temp/stds/max_vals
                        else:
                            temp = 3.0*temp/stds/max_vals

                    else:
                        temp = (temp - means)/stds

                    input_frame[..., n] = temp

                # CNN feature vectors

                temp_vec = model.predict([input_frame])
                FEATURE_VEC[ix, iy, l, :] = temp_vec[0, :]

In [95]:
save_dir

'/glade/work/ksha/NCAR/'